In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers,models

In [2]:
data=pd.read_csv('/content/insurance_data (2).csv')
data.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [3]:
x=data.drop('bought_insurance',axis=1)
y=data['bought_insurance']

In [5]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [6]:
x_train_scaled=x_train.copy()
x_train_scaled['age']=x_train['age']/100
x_test_scaled=x_test.copy()
x_test_scaled['age']=x_test['age']/100

In [7]:
model=keras.Sequential([
    keras.layers.Dense(1,input_shape=(2,),activation='sigmoid',kernel_initializer='ones', bias_initializer='zeros')

                       ])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
model.compile(optimizer='adam',

              loss='binary_crossentropy',
              metrics=['accuracy']
              )

In [11]:
model.fit(x_train_scaled,y_train,epochs=1500,verbose=0)

In [12]:
model.evaluate(x_test_scaled,y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 1.0000 - loss: 0.4101


[0.4100762903690338, 1.0]

In [13]:
model.predict(x_test_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step


array([[0.7197772 ],
       [0.68142545],
       [0.72502047],
       [0.31698042],
       [0.48091877],
       [0.32266816]], dtype=float32)

In [14]:
y_test

,bought_insurance
9,1
25,1
8,1
21,0
0,0
12,0


In [15]:
coef,intersept=model.get_weights()
coef,intersept


(array([[2.6146636],
        [0.795908 ]], dtype=float32),
 array([-1.447496], dtype=float32))

In [17]:
def sigmoid(x):
  import math
  return 1/(1+math.exp(-x))
print(sigmoid(18))

0.9999999847700205


In [18]:
def prediction(age,affor):
  sum_weights=coef[0]*age+coef[1]*affor+intersept
  return sigmoid(sum_weights)
prediction(.47,1)

/tmp/ipython-input-409418760.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 1/(1+math.exp(-x))


0.6404467675195856

In [19]:
def sigmoid_numpy(x):
  return 1/(1+np.exp(-x))
sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [20]:
def log_loss(y_true,y_predicted):
  epsilon=1e-15
  y_predicted_new=[max(i,epsilon) for i in y_predicted]
  y_predicted_new=[min(i,epsilon) for i in y_predicted_new]
  y_predicted_new=np.array(y_predicted_new)
  return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))


In [54]:
class NN:
  def __init__(self):
    self.w1=1
    self.w2=1
    self.bais=0
  def fit(self,x,y,epochs,loss_thresold):
    self.w1,self.w2,self.bais=self.gradient_descent(x['age'],x['affordibility'],y,epochs,loss_thresold)
    print(f'Final weights and bias: w1:{self.w1},w2:{self.w2},bais:{self.bais}')
  def predict(self, x_test):
        weighted_sum = self.w1*x_test['age'] + self.w2*x_test['affordibility'] + self.bais
        return sigmoid_numpy(weighted_sum)

  def gradient_descent(self, age,affordability, y_true, epochs, loss_thresold):
        w1 = w2 = 1
        bais = 0
        rate = 0.5
        n = len(age)
        for i in range(epochs):
            weighted_sum = w1 * age + w2 * affordability + bais
            y_predicted = sigmoid_numpy(weighted_sum)
            loss = log_loss(y_true, y_predicted)

            w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true))
            w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true))

            bias_d = np.mean(y_predicted-y_true)
            w1 = w1 - rate * w1d
            w2 = w2 - rate * w2d
            bais = bais - rate * bias_d

            if i%50==0:
                print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bais}, loss:{loss}')

            if loss<=loss_thresold:
                print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bais}, loss:{loss}')
                break

        return w1, w2, bais

In [55]:
customModel =NN()
customModel.fit(x_train_scaled, y_train, epochs=500, loss_thresold=0.4631)

Epoch:0, w1:0.9736899318847281, w2:0.931388810977659, bias:-0.11748951666770448, loss:17.26938819745534
Epoch:50, w1:1.524279872239132, w2:0.8822187836689879, bias:-1.1310088596841466, loss:17.26938819745534
Epoch:100, w1:2.2281870363373972, w2:1.021207570480777, bias:-1.5219348336892933, loss:17.26938819745534
Epoch:150, w1:2.8788802020633866, w2:1.0918966282424585, bias:-1.8376731064036336, loss:17.26938819745534
Epoch:200, w1:3.473286412049912, w2:1.1354958313845407, bias:-2.109494376761035, loss:17.26938819745534
Epoch:250, w1:4.014235251285109, w2:1.168307211546194, bias:-2.3515205001257873, loss:17.26938819745534
Epoch:300, w1:4.506491403088423, w2:1.1967920767703706, bias:-2.570834600613761, loss:17.26938819745534
Epoch:350, w1:4.955154772161112, w2:1.2234470550440064, bias:-2.7714756232427904, loss:17.26938819745534
Epoch:400, w1:5.365041991551451, w2:1.2491715720776855, bias:-2.956078966702517, loss:17.26938819745534
Epoch:450, w1:5.740481994774169, w2:1.2742348838393918, bias

In [56]:
coef, intersept

(array([[2.6146636],
        [0.795908 ]], dtype=float32),
 array([-1.447496], dtype=float32))

In [57]:
x_test_scaled

,age,affordibility
9,0.61,1
25,0.54,1
8,0.62,1
21,0.26,0
0,0.22,1
12,0.27,0


In [58]:
customModel.predict(x_test_scaled)

,0
9,0.848738
25,0.785705
8,0.856377
21,0.154336
0,0.343904
12,0.162438


In [59]:
model.predict(x_test_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


array([[0.7197772 ],
       [0.68142545],
       [0.72502047],
       [0.31698042],
       [0.48091877],
       [0.32266816]], dtype=float32)